<a href="https://colab.research.google.com/github/harsv689/ISS-Lab-1_old/blob/master/court_cases_analysis_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [ ]:
# !sudo apt-get install python3-pip
# !sudo python3 -m pip install ipykernel
# !pip3 install pandas

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from google.colab import drive


#### Function to read the csv files

In [ ]:
def read_csv(filename):
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            print(row)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
# read_csv("./judges_clean.csv")


# Converting the CSV files to Pandas dataframe

In [ ]:
df_judges = pd.read_csv("/content/drive/Shareddrives/Precog/Precog/judges_clean.csv")
df_judges.head(100)


,ddl_judge_id,state_code,dist_code,court_no,judge_position,female_judge,start_date,end_date
0,1,1,1,1,chief judicial magistrate,0 nonfemale,20-09-2013,20-02-2014
1,2,1,1,1,chief judicial magistrate,0 nonfemale,31-10-2013,20-02-2014
2,3,1,1,1,chief judicial magistrate,0 nonfemale,21-02-2014,31-05-2016
3,4,1,1,1,chief judicial magistrate,0 nonfemale,01-06-2016,06-06-2016
4,5,1,1,1,chief judicial magistrate,0 nonfemale,06-06-2016,07-07-2018
...,...,...,...,...,...,...,...,...
95,96,1,1,4,civil judge junior division,0 nonfemale,01-03-2007,01-06-2015
96,97,1,1,4,civil judge junior division,0 nonfemale,01-06-2015,31-08-2017
97,98,1,1,4,civil judge junior division,0 nonfemale,04-06-2018,NaN
98,99,1,1,4,civil judge senior division,0 nonfemale,01-03-2007,04-06-2012


In [ ]:
df_judges.head(10)
print("The number of unique type of gender of judges in the data : ")
df_judges["female_judge"].unique()


The number of unique type of gender of judges in the data : 


array(['0 nonfemale', '1 female', '-9998 unclear', nan], dtype=object)

In [ ]:
print("The count of those types : ")
df_judges["female_judge"].value_counts()


The count of those types : 


0 nonfemale      67540
1 female         27202
-9998 unclear     3735
Name: female_judge, dtype: int64

In [ ]:
#Removing the -9998 unclear rows
df_judges=df_judges[df_judges["female_judge"] != "-9998 unclear"]
df_judges["female_judge"].unique()


array(['0 nonfemale', '1 female', nan], dtype=object)

In [ ]:
# Converting the strings to numbers
# "0 nonfemale" & to 0
# "1 female" to 1
# df_judges.loc.__setitem__((slice(None),("female_judge","1 female")),0)
df_judges["female_judge"]=(df_judges["female_judge"]=="1 female").astype(int)
df_judges.head(10)
# preprocessed data for finding out female or not

,ddl_judge_id,state_code,dist_code,court_no,judge_position,female_judge,start_date,end_date
0,1,1,1,1,chief judicial magistrate,0,20-09-2013,20-02-2014
1,2,1,1,1,chief judicial magistrate,0,31-10-2013,20-02-2014
2,3,1,1,1,chief judicial magistrate,0,21-02-2014,31-05-2016
3,4,1,1,1,chief judicial magistrate,0,01-06-2016,06-06-2016
4,5,1,1,1,chief judicial magistrate,0,06-06-2016,07-07-2018
5,6,1,1,1,chief judicial magistrate,1,09-07-2018,NaN
6,7,1,1,1,civil judge junior division,0,01-06-2011,09-06-2013
7,8,1,1,1,civil judge junior division,0,06-06-2011,08-06-2014
8,9,1,1,1,civil judge junior division,0,11-06-2012,31-05-2015
9,10,1,1,1,civil judge junior division,0,29-08-2013,08-06-2014


In [ ]:
print("Percentage Of Female Judge In India : ")
value=df_judges["female_judge"].value_counts()
male=value[0]  # assuming the unclear gender to be maintaing the same ratio. The national average is given by :-
female=value[1]
national_average=female/(male+female)
print(national_average*100)


Percentage Of Female Judge In India : 
28.71135598408326


In [ ]:
print("The count of those types : ") # Trying to find out effect of position on female representation
df_judges["judge_position"].value_counts()
# male=value[0]  # assuming the unclear gender to be maintaing the same ratio. The national average is given by :-
# female=value[1]
# national_average=female/male
# print(national_average)


The count of those types : 


district and sessions court                       19771
chief judicial magistrate                         14224
civil judge senior division                       13438
civil judge junior division                        9726
civil court                                        4645
                                                  ...  
j.m 1st class cum addl. munsif-ii                     1
presiding officer ftc i                               1
v-addl.munsif- iii                                    1
adhoc addl. distt. and sess. judge - ii               1
court no. 5 of j.m 1st class, muzaffarpur east        1
Name: judge_position, Length: 565, dtype: int64

### Checking the cases that were closed within a year

In [ ]:
closed_within_year=0
not_closed_within_year=0
df_judges["start_date"]=pd.to_datetime(df_judges["start_date"])
df_judges["end_date"]=pd.to_datetime(df_judges["end_date"])

df_judges.head()
temp_df=(df_judges["end_date"]-df_judges["start_date"])
temp_df=temp_df.dt.days
temp_df=temp_df[temp_df<=1500] #it was giving values upto 8000 so I limited it to 1500



closed_within_year=temp_df[temp_df<=365].count()
not_closed_within_year=temp_df[temp_df>365].count()


print("Closed within a year : ",closed_within_year)
print("Not closed within a year : ",not_closed_within_year)


Closed within a year :  44120
Not closed within a year :  30932


### Count the number of judges in each state


In [ ]:
# Count the number of judges in each state
judgesperstate=df_judges["state_code"].value_counts()
df_judgesperstate=pd.DataFrame(judgesperstate)

df_judgesperstate["State_code"]=df_judgesperstate.index
df_judgesperstate.rename(columns={"state_code":"Number_of_judges"},inplace=True)
df_judgesperstate.reset_index(drop=True,inplace=True)
df_judgesperstate["State_name"]="temp"

print(df_judgesperstate)


    Number_of_judges  State_code State_name
0              21987           1       temp
1              13999          13       temp
2               6440           3       temp
3               5583          23       temp
4               5390           9       temp
5               4949          17       temp
6               4121           8       temp
7               3655          10       temp
8               3492           2       temp
9               3060           4       temp
10              2677          16       temp
11              2568          29       temp
12              2321          14       temp
13              2283           7       temp
14              2034          11       temp
15              1921          22       temp
16              1550           6       temp
17              1457          26       temp
18              1078          15       temp
19              1032          18       temp
20               899          12       temp
21               897           5

In [ ]:
# Getting the state codes 

df_statecodes=pd.read_csv("./keys/cases_state_key.csv")

#storing only on basis of 2018 codes
df_statecodes=df_statecodes[df_statecodes["year"]==2018]
df_statecodes.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
df_statecodes.sort_values(by="state_code",inplace=True)
df_statecodes.reset_index(drop=True,inplace=True)
print(df_statecodes)


    state_code         state_name
0            1        Maharashtra
1            2     Andhra Pradesh
2            3          Karnataka
3            4             Kerala
4            5   Himachal Pradesh
5            6              Assam
6            7          Jharkhand
7            8              Bihar
8            9          Rajasthan
9           10         Tamil Nadu
10          11             Orissa
11          12  Jammu and Kashmir
12          13      Uttar Pradesh
13          14            Haryana
14          15        Uttarakhand
15          16        West Bengal
16          17            Gujarat
17          18       Chhattisgarh
18          19            Mizoram
19          20            Tripura
20          21          Meghalaya
21          22             Punjab
22          23     Madhya Pradesh
23          24             Sikkim
24          25            Manipur
25          26              Delhi
26          27         Chandigarh
27          29          Telangana
28          30

In [ ]:
#assigning the state names to the state codes
for i in range(len(df_judgesperstate)):
    df_judgesperstate.loc[i,"State_name"]=df_statecodes.loc[i,"state_name"]

df_judgesperstate.drop("State_code",axis=1,inplace=True)
print(df_judgesperstate)


    Number_of_judges         State_name
0              21987        Maharashtra
1              13999     Andhra Pradesh
2               6440          Karnataka
3               5583             Kerala
4               5390   Himachal Pradesh
5               4949              Assam
6               4121          Jharkhand
7               3655              Bihar
8               3492          Rajasthan
9               3060         Tamil Nadu
10              2677             Orissa
11              2568  Jammu and Kashmir
12              2321      Uttar Pradesh
13              2283            Haryana
14              2034        Uttarakhand
15              1921        West Bengal
16              1550            Gujarat
17              1457       Chhattisgarh
18              1078            Mizoram
19              1032            Tripura
20               899          Meghalaya
21               897             Punjab
22               438     Madhya Pradesh
23               324             Sikkim


**Female Judges Representation In different States over the Years (2014-2019)**

In [63]:
whole_data = pd.read_csv("/content/drive/Shareddrives/Precog/Precog/judges_clean.csv")
# Removing the Unclear data
whole_data=whole_data[whole_data["female_judge"] != "-9998 unclear"]
whole_data["female_judge"]=(whole_data["female_judge"]=="1 female").astype(int)
# whole_data.drop["dist_code","court_no","ddl_judge_id","judge_position","start_date","end_date"]
whole_data.drop(whole_data.columns[[0,2,3,4]],axis=1,inplace=True)
whole_data.head(100)
# whole_data.max()
# whole_data.min()

,state_code,female_judge,start_date,end_date
0,1,0,20-09-2013,20-02-2014
1,1,0,31-10-2013,20-02-2014
2,1,0,21-02-2014,31-05-2016
3,1,0,01-06-2016,06-06-2016
4,1,0,06-06-2016,07-07-2018
...,...,...,...,...
95,1,0,01-03-2007,01-06-2015
96,1,0,01-06-2015,31-08-2017
97,1,0,04-06-2018,NaN
98,1,0,01-03-2007,04-06-2012


In [90]:
data_2014=pd.DataFrame()
data_2014=whole_data[whole_data["end_date"] >= ("01-01-2014")]
data_2014=data_2014[data_2014["start_date"] <= ("01-01-2014")]
data_2015=pd.DataFrame()
data_2015=whole_data[whole_data["end_date"] >= ("01-01-2015")]
data_2015=data_2015[data_2015["start_date"] <= ("01-01-2015")]
data_2016=pd.DataFrame()
data_2016=whole_data[whole_data["end_date"] >= ("01-01-2016")]
data_2016=data_2016[data_2016["start_date"] <= ("01-01-2016")]
data_2017=pd.DataFrame()
data_2017=whole_data[whole_data["end_date"] >= ("01-01-2017")]
data_2017=data_2017[data_2017["start_date"] <= ("01-01-2017")]
data_2018=pd.DataFrame()
data_2018=whole_data[whole_data["end_date"] >= ("01-01-2018")]
data_2018=data_2018[data_2018["start_date"] <= ("01-01-2018")]
data_2019=pd.DataFrame()
data_2019=whole_data[whole_data["end_date"] >= ("01-01-2019")]
data_2019=data_2019[data_2019["start_date"] <= ("01-01-2019")]
# data_2019.count()

In [111]:
# Analysing 2014 female judge represenatation state wise


# data_2014.max()
# state_code              30
# female_judge             1
# start_date      01-01-2014
# end_date        31-12-2015

# data_2014.min()
# state_code               1
# female_judge             0
# start_date      01-01-1995
# end_date        01-01-2015

# data_2014["state_code"].value_counts() 
jdg_female=pd.DataFrame()
jdg_female=data_2014[data_2014["female_judge"] == 1]
temp_2014=jdg_female["state_code"].value_counts()
jdg_2014_f=pd.DataFrame(temp_2014)
jdg_2014_f["State_Code"]=jdg_2014_f.index
jdg_2014_f.rename(columns={"state_code":"Number_of_Female_Judges"},inplace=True)
jdg_2014_f.reset_index(drop=True,inplace=True)
jdg_2014_f.head(10)

,Number_of_Female_Judges,State_Code
0,56,1
1,37,16
2,32,23
3,22,3
4,22,10
5,14,15
6,12,12
7,9,17
8,9,29
9,8,2


In [130]:
jdg_total = data_2014["state_code"].value_counts()
jdg_2014=pd.DataFrame(jdg_total)
jdg_2014["State_Code"]=jdg_2014.index
jdg_2014.rename(columns={"state_code":"Number_of_Judges"},inplace=True)
jdg_2014.reset_index(drop=True,inplace=True)
jdg_2014["State_name"]="temp"
jdg_2014=pd.merge(jdg_2014,jdg_2014_f,how="outer",on="State_Code")
jdg_2014["Number_of_Female_Judges"]=jdg_2014['Number_of_Female_Judges'].fillna(0)
jdg_2014 = jdg_2014.astype({'Number_of_Female_Judges':'int'})
state_code_2014=pd.read_csv("/content/drive/Shareddrives/Precog/Precog/Key/cases_state_key.csv")
state_code_2014=state_code_2014[state_code_2014["year"]==2014]
state_code_2014.drop(["year","pc11_state_name","pc11_state_id"],axis=1,inplace=True)
state_code_2014.sort_values(by="state_code",inplace=True)
state_code_2014.reset_index(drop=True,inplace=True)
for i in range (len(jdg_2014)):
  jdg_2014.loc[i,"State_name"]=state_code_2014.loc[i,"state_name"]

jdg_2014.drop("State_Code",axis=1,inplace=True)
jdg_2014



,Number_of_Judges,State_name,Number_of_Female_Judges
0,231,Maharashtra,56
1,150,Andhra Pradesh,32
2,95,Karnataka,7
3,93,Kerala,37
4,82,Himachal Pradesh,12
5,79,Assam,22
6,54,Jharkhand,22
7,37,Bihar,4
8,27,Rajasthan,9
9,26,Tamil Nadu,2
